# WebocketClientWorker Tutorial

This tutorial is a 2 notebook tutorial. The partner notebook is the notebook entitled `WebsocketServerWorker.ipynb` and is in the same folder as this notebook. You should execute this notebook AFTER you have executed the other one.

In that tutorial, we'll demonstrate how to launch a WebsocketWorker server which will listen for PyTorch commands over a socket connection. In this tutorial, the two workers are connected via a socket connection on the localhost network.

If you'd prefer to download this notebook and run it locally, you can do so via the `Download .ipynb` in the `File` dropdown field in this Google colab environment. 

Setup instructions: https://research.google.com/colaboratory/local-runtimes.html


# Step -1: Copy This Notebook

Go up to File -> Save A Copy in Drive

This will let you execute the notebook (it won't let you execute this one by default)

# Step 0: Install Dependencies

In [0]:
# If not using local runtimes, uncomment out the following snippets where necessary.

#! rm -rf /content/PySyft
#! git clone https://github.com/OpenMined/PySyft.git
# http://pytorch.org/
#from os import path
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag

#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
#cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
#accelerator = cuda_output[0] if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
#!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl
 
import torch

#!cd PySyft; pip3 install -r requirements.txt; pip3 install -r requirements_dev.txt; python3 setup.py install

import os
import sys

#module_path = '/content/PySyft'  # You want './PySyft' to be on your sys.path
#if module_path not in sys.path:
#    sys.path.append(module_path)

fatal: destination path 'PySyft' already exists and is not an empty directory.
running install
running bdist_egg
running egg_info
writing syft.egg-info/PKG-INFO
writing dependency_links to syft.egg-info/dependency_links.txt
writing requirements to syft.egg-info/requires.txt
writing top-level names to syft.egg-info/top_level.txt
reading manifest file 'syft.egg-info/SOURCES.txt'
writing manifest file 'syft.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/test
creating build/bdist.linux-x86_64/egg/test/core
copying build/lib/test/core/utils_test.py -> build/bdist.linux-x86_64/egg/test/core
copying build/lib/test/core/workers_test.py -> build/bdist.linux-x86_64/egg/test/core
copying build/lib/test/core/hooks_test.py -> build/bdist.linux-x86_64/egg/test/core
copying build/lib/test/core/__init__.py -> build/bdist.linux-x86_64/egg/test/core
copying buil

In [0]:
# Check sys.path
#sys.path

# Step 1: Hook Torch and Create Local Worker

In this step, we hook PyTorch and initialize within the hook a client SocketWorker.

In [0]:
import syft
from syft.workers.websocket_server import WebsocketServerWorker

hook = syft.TorchHook(torch)

local_worker = WebsocketServerWorker(
                            host='localhost',
                            hook=hook,
                            id=0,
                            port=8182,
                            log_msgs=True,
                            verbose=True)

ModuleNotFoundError: ignored

# Step 2: Create Pointer to Remote Socket Worker

In order to interact with a foreign worker over a socket connection, we need to create a pointer to it containing information on how to contact it. We set the is_pointer=True to signify that this Python object is not in fact a worker in and of itself but that it is merely a pointer to one over the network. We then inform our local worker about this pointer.

In [0]:
hook = syft.TorchHook(torch, local_worker=local_worker)

from syft.workers.websocket_client import WebsocketClientWorker

remote_client = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id=2, 
                            port=8181)

hook.local_worker.add_worker(remote_client)

Attaching Pointer to Socket Worker...


# Step 3: Create Tensors & Send To The Worker

In [0]:
x = syft.FixedPrecisionTensor([1,3,5,7,9]).share(remote_client)

In [0]:
x2 = syft.FixedPrecisionTensor([2,4,6,8,10]).share(remote_client)

In [0]:
x

FloatTensor[_PointerTensor - id:4129544610 owner:0 loc:2 id@loc:79320779676]

In [0]:
x2

FloatTensor[_PointerTensor - id:8144291470 owner:0 loc:2 id@loc:19397969248]

# Step 4: Execute Operations Like Normal

In [0]:
y = x + x2 + x

# Step 5: Get Results

In [0]:
y

FloatTensor[_PointerTensor - id:1340811713 owner:0 loc:2 id@loc:1136221544]

In [0]:
y.get()


  3
  6
  9
 12
 14
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]

In [0]:
y


  3
  6
  9
 12
 14
[syft.core.frameworks.torch.tensor.FloatTensor of size 5]